In [121]:
%run /home/jovyan/data_loading_tools.ipynb
%run /home/jovyan/plotting_tools.ipynb

In [122]:
# Client täytyy käynnistää ekana
client = start_client()

In [123]:
# Alustetaan datan hakua
parameters = ['Temperature']
time_range = 10                   # Määrittää kuinka monelta päivältä dataa haetaan tästä hetkestä eteenpäin
model = 'EC'

delayed_data_ec = get_model_data_delay(parameters, time_range, model)
EC_params = merge_model_delayed(client, delayed_data_ec)

Done!


In [124]:
EC_params.head().compute()

<xarray.Dataset>
Dimensions:            (time: 5, y: 5, x: 5)
Coordinates:
  * time               (time) datetime64[ns] 2024-09-11 ... 2024-09-11T04:00:00
  * y                  (y) float32 -3.883e+06 -3.876e+06 ... -3.854e+06
  * x                  (x) float32 -1.01e+06 -1.003e+06 ... -9.806e+05
    lat                (y, x) float32 52.7 52.72 52.73 ... 52.98 53.0 53.01
    lon                (y, x) float32 5.421 5.523 5.625 ... 5.519 5.621 5.724
Data variables:
    crs                int16 -32767
    air_temperature_4  (time, y, x) float32 284.3 284.7 285.7 ... 283.9 284.3
Attributes:
    Conventions:  CF-1.6
    title:        <title>
    institution:  fmi.fi
    source:       <producer>

In [125]:
# Satunnaista testailua serveriltä
#print(EC_params['air_temperature_4'].max(dim="time").values.compute())
#print(EC_params['air_temperature_4'].groupby("time.hour").mean().values.compute())
#print(EC_params['air_temperature_4'].resample(time="1D").compute())

# Liukuva keskiarvo menneiden aika-askelten perusteella. --> Alussa Nan-arvoja
#print(EC_params['air_temperature_4'].rolling(time=5, center=False).mean().compute())

# Liukuva keskiarvo menneiden aika-askelten perusteella. --> lopussa Nan-arvoja
#print(EC_params['air_temperature_4'][::-1].rolling(time=5, center=False).mean()[::-1].compute())


chart = EC_params['air_temperature_4'].isel(time=1).compute()
#print(chart.head().compute())

# Meteorologit voi aloittaa tekemisen tästä kohtaa. Ylimpänä oleva "solu" pitää ajaa, jotta saadaan metodit käyttöön

# Tästä eteenpäin tehdään visualisointeja

In [126]:
# Plotataan suoraan serveriltä kelluva keskiarvo. Laskeminen tapahtuu tässä solussa funktion sisällä
def draw(parameter, t):    
    # Luetaan ja käsitellään dataa
    # Otetaan data ulos serveriltä ns. nurinpäin lopusta alkuun
    chart = EC_params['air_temperature_4'][::-1].compute()
    # Lasketaan kelluva keskiarvo 5h ikkunalle ja käännetään data takaisin alkuperäiseen järjestykseen
    chart = chart.rolling(time=5, center=False).mean()[::-1]
    # Otetaan datasta plotattavaksi t-indeksillä oleva aika-askel
    chart = chart.isel(time=t)
    # Yksikkömuutos ja muuta mukavaa
    chart = chart - 273.15
    cmap = 'nipy_spectral'
    vmin = -40
    vmax = 40

    # Luodaan karttapohja ja kuva
    axx = get_map_layout()

    # Plotataan itse data
    plot_map_xarray(chart, vmin, vmax, cmap, axx)

In [127]:
# Luodaan widget
widgets.interact(draw, parameter=['air_temperature_4'], t=(0, len(EC_params['time'].compute()) - 1))

interactive(children=(Dropdown(description='parameter', options=('air_temperature_4',), value='air_temperature…

<function __main__.draw(parameter, t)>